<a href="https://colab.research.google.com/github/MohitDhungana/duplicate_question_detection/blob/master/quora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import itertools
import datetime


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls '/content/drive/My Drive'



 53678.jpg				'How to get started with Drive.pdf'
'6th sem.zip (Unzipped Files)'		 java_all_video.rar
'Colab Notebooks'			 mid-term-report-computer.gdoc
 final.rar				 q_quora.csv
 final.zip				 stock_price.csv
 flower.csv				 train.csv
 GoogleNews-vectors-negative300.bin.gz


In [0]:
df = pd.read_csv('/content/drive/My Drive/train.csv')

stops = set(stopwords.words('english'))

def text_to_word_list(text):
#     gets a whole question inside text variable on which  preprocessing is done and then the question is splitted into word indices and returned
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [0]:

# Prepare embedding

vocabulary = dict() 
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz', binary=True)

questions_cols = ['question1', 'question2']

# Iterate over the questions(i.e. 'question 1', 'question2') of the training sets provided 
# get the data from dataset(df) in  variable dataset
for dataset in [df]:
#   now iterate through the rows of the dataset using  index as iterator (i.e. goes through the row having particular index at a time) 
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions within the current  row
        for question in questions_cols:
            que2no = []  # que2no ->  numbers representation of the cureently being processed question
            for word in text_to_word_list(row[question]):

                # Check for unwanted words i.e mainly stopwords
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    que2no.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    que2no.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, que2no)
            
            

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [0]:
embedding_dim = 300
# creating an embedding matrix whose length is one more than vocabulary and dimension is 300
embedding_matrix = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  
embedding_matrix[0] = 0  # So that the zeroth place remain empty 

# (Build the embedding matrix) Assigning the word2vec embedding for each words of our vocabulary  
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embedding_matrix[index] = word2vec.word_vec(word)

In [0]:
from keras.preprocessing.sequence import pad_sequences

max_seq_length = max(df.question1.map(lambda x: len(x)).max(),
                     df.question2.map(lambda x: len(x)).max()
                    )
# questions_cols = ['question1', 'question2']
# Split to train validation

validation_size = 40000
training_size = len(df) - validation_size

X=df[questions_cols]
Y=df['is_duplicate']


x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size = validation_size)
                 
  # Split to dicts
x_train = {'left': x_train.question1, 'right': x_train.question2}
x_test = {'left': x_test.question1, 'right': x_test.question2}

# Convert labels to their numpy representations
y_train=y_train.values
y_test=y_test.values


# zero padding

for dataset, side in itertools.product([x_train,x_test],['left','right']):
  dataset[side] = pad_sequences(dataset[side], maxlen = max_seq_length)
